In [1]:
import cv2
from PIL import Image 
import matplotlib 
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage

# Question 1

In [2]:
fig = plt.figure()
whiteblackimage = 255 * np.ones(shape=[500, 500, 3]) 
cv2.rectangle(whiteblackimage, pt1=(200,200), pt2=(300,300), color=(0,0,0), thickness=-1)
whiteblackImage = np.array(whiteblackimage)

In [16]:
magnitudes = []
sigmas = np.linspace(20, 100, 200)
for sigma in sigmas:
    print(sigma)
    laplacian = ndimage.gaussian_laplace(whiteblackimage, sigma=sigma)
    # Get normalized laplacian
    norm_laplacian = laplacian * (sigma ** 2)
    magnitudes.append(np.max(norm_laplacian))

20.0
20.402010050251256
20.804020100502512
21.20603015075377
21.608040201005025
22.01005025125628
22.412060301507537
22.814070351758794
23.21608040201005
23.618090452261306
24.020100502512562
24.42211055276382
24.824120603015075
25.22613065326633
25.628140703517587
26.030150753768844
26.4321608040201
26.834170854271356
27.236180904522612
27.63819095477387
28.040201005025125
28.44221105527638
28.844221105527637
29.246231155778894
29.64824120603015
30.050251256281406
30.452261306532662
30.85427135678392
31.256281407035175
31.65829145728643
32.06030150753769
32.462311557788944
32.8643216080402
33.266331658291456
33.66834170854271
34.07035175879397
34.472361809045225
34.87437185929648
35.27638190954774
35.678391959798994
36.08040201005025
36.48241206030151
36.88442211055276
37.286432160804026
37.688442211055275
38.09045226130654
38.49246231155779
38.89447236180905
39.2964824120603
39.69849246231156
40.10050251256281
40.502512562814076
40.904522613065325
41.30653266331659
41.70854271356784


In [23]:
max_mag = np.argmax(np.array(magnitudes))
print(magnitudes[max_mag])
fig = plt.figure()
plt.plot(sigmas, magnitudes)
print(sigmas[np.argmax(magnitudes)])
fig.savefig("Q1plots.png")

183.07166263121093
40.502512562814076


# Question 3

In [3]:
image1 = Image.open("Q3/1.jpg")
plt.imshow(np.array(image1), cmap="gray")
gray1 = np.array(image1)
print(gray1.shape)

(286, 230)


In [8]:
def center_crop(image, m, n, tau):
    center_x, center_y = image.shape[0] // 2, image.shape[1] // 2
    return image[center_x - m*tau//2: center_x+ m*tau//2, center_y - n*tau//2: center_y + n*tau//2]

def construct_HOG(orientation, gradient, m, n, tau):
    bins = [(-15, 15), (15, 45), (45, 75), (75, 105), (105, 135), (135, 165)]
    matrix = np.zeros((m,n,len(bins)))
    bincounts = np.zeros((m, n, len(bins)))
    for i in range(orientation.shape[0]):
        for j in range(orientation.shape[1]):
            i_m = i // tau
            j_n = j // tau
            # handle the case when orientation is larger than 165
            if orientation[i][j] >= 165:
                matrix[i_m][j_n][0] += gradient[i][j]
                bincounts[i_m][j_n][0] += 1
                continue
            for k in range(len(bins)):
                if bins[k][0] <= orientation[i][j] < bins[k][1]:
                    matrix[i_m][j_n][k] += gradient[i][j]
                    bincounts[i_m][j_n][k] += 1
                    break
    return matrix, bincounts

def compute_image_gradient_and_threshold(image, tau, epsilon):
    # center crop an image
    dim = image.shape
    print(dim)
    m, n = dim[0] // tau, dim[1] // tau
    print(f"M, N:{m, n}")
    image = center_crop(image, m, n, tau)
    print(image.shape)
    # compute gradients of x and y directions
    g_x = cv2.Sobel(image, cv2.CV_64F, 1, 0)
    g_y = cv2.Sobel(image, cv2.CV_64F, 0, 1)
    # compute the magnitude of gradients
    gradient = np.sqrt(np.square(g_x) + np.square(g_y))
    orientation = np.arctan2(g_y, g_x) * (180 / np.pi)
    # Thresholding gradient magnitude
    gradient = np.where(gradient < epsilon, 0, gradient)
    # Convert the negative orientation 
    orientation = np.where(orientation < 0, 180+orientation, orientation)
    # construct HOG
    print(orientation.shape)
    gradient_mtx, bincounts = construct_HOG(orientation, gradient, m, n, tau)
    return gradient, orientation, gradient_mtx, bincounts

def generate_quiver_plot(gradient_mtx, image, tau, name):
    m, n, bincount = gradient_mtx.shape
    print(m, n)
    bins = [(-15, 15), (15, 45), (45, 75), (75, 105), (105, 135), (135, 165)]
    # X and Y are 2D
    X, Y = np.meshgrid(np.linspace(0, n*tau, n), np.linspace(0, m*tau, m))
    fig, ax = plt.subplots()
    ax.imshow(image, cmap="gray")
    # Create a quiver plot for each of the bin
    # Make U and V 2D as well => have to loop over each bin 
    # Obtain the U and V with sin and cos functions
    for k in range(bincount):
        mean_angle = (bins[k][0] + bins[k][1])/2 * np.pi/180
        # Compute x component by multiplying |gradient|cos(\theta)
        U = np.sin(mean_angle) * gradient_mtx[:, :, k]
        # Compute y component by multiplying |gradient|sin(\theta)
        V = np.cos(mean_angle) * gradient_mtx[:, :, k]
        #ax.quiver(X, Y, U, V, color="blue", pivot="middle")
        ax.quiver(X, Y, U, V, color="blue", linewidth=.5, headlength=0, 
                  headwidth=1, headaxislength=0, pivot="middle")
    # save fig
    fig.savefig(f"images/{name}_quiver.png")
    # show figure
    fig.show()


In [9]:
gradient, orientation, gradient_mtx, bincounts = compute_image_gradient_and_threshold(gray1, 8, 0.01)

(286, 230)
M, N:(35, 28)
(280, 224)
(280, 224)


In [10]:
generate_quiver_plot(gradient_mtx, gray1, 8, "Q3_1")
generate_quiver_plot(bincounts, gray1, 8, "Q3_1_counts")

35 28
35 28


In [11]:
def get_normalized_histogram(gradient_mtx, name, block_size=2, e=0.0001, bin_count=6):
    m, n, _ = gradient_mtx.shape
    normalized_histogram = []
    txt_array = []
    for i in range(m - block_size + 1):
        for j in range(n-block_size+ 1):
            h_i = gradient_mtx[i:i+block_size, j:j+block_size].flatten()
            normalized_descriptor =  h_i/np.sqrt(np.sum(np.square(h_i)) + np.square(e))
            normalized_histogram.append(normalized_descriptor)
            txt_array.append(normalized_descriptor)
    normalized_histogram = np.array(normalized_histogram).reshape((m-1, n-1, block_size*block_size*bin_count))
    np.savetxt(f"./txtfiles/{name}.txt", np.array(txt_array), delimiter=",")
    return normalized_histogram

In [34]:
normalized = get_normalized_histogram(gradient_mtx, "Q3_1")

In [12]:
image2 = Image.open("Q3/2.jpg")
plt.imshow(np.array(image2), cmap="gray")
image2 = np.array(image2)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)


In [13]:
gradient2, orientation2, gradient_mtx2, bincounts2 = compute_image_gradient_and_threshold(gray2, 8, 0.01)


(448, 320)
M, N:(56, 40)
(448, 320)
(448, 320)


In [14]:
generate_quiver_plot(gradient_mtx2, image2, 8, "Q3_2")
generate_quiver_plot(bincounts2, image2, 8, "Q3_2_count")
normalized = get_normalized_histogram(gradient_mtx2, "Q3_2")

56 40
56 40


In [9]:
for cell in [4, 6, 8, 12, 20]:
    for epsilon in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]:
        gradient, orientation, gradient_mtx, bincounts = compute_image_gradient_and_threshold(gray1, cell, epsilon)
        generate_quiver_plot(gradient_mtx, gray1, cell, f"images/Q3_1_{cell}_{epsilon}")
        generate_quiver_plot(bincounts, gray1, cell, f"images/Q3_1_counts_{cell}_{epsilon}")
        gradient2, orientation2, gradient_mtx2, bincounts2 = compute_image_gradient_and_threshold(gray2, cell, epsilon)
        generate_quiver_plot(gradient_mtx2, image2, cell, f"images/Q3_2_{cell}_{epsilon}")
        generate_quiver_plot(bincounts2, image2, cell, f"images/Q3_2_count_{cell}_{epsilon}")

(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80
(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80
(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80
(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80
(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80


<ipython-input-5-126aa25ca744>:56: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


(286, 230)
M, N:(71, 57)
(284, 228)
(284, 228)
71 57
71 57
(448, 320)
M, N:(112, 80)
(448, 320)
(448, 320)
112 80
112 80
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(47, 38)
(282, 228)
(282, 228)
47 38
47 38
(448, 320)
M, N:(74, 53)
(444, 318)
(444, 318)
74 53
74 53
(286, 230)
M, N:(35, 28)
(280, 224)
(280, 224)
35 28
35 28
(448, 320)
M, N:(56, 40)
(448, 320)
(448, 320)
56 40
56 40
(286, 230)
M, N:(35, 28)
(280, 224)
(280, 224)
35 28


In [15]:
image3 = Image.open("Q3/3.jpg")
#plt.imshow(np.array(image2), cmap="gray")
image3 = np.array(image3)
gray3 = cv2.cvtColor(image3, cv2.COLOR_BGR2GRAY)
gradient3, orientation3, gradient_mtx3, bincounts3 = compute_image_gradient_and_threshold(gray3, 8, 0.01)
generate_quiver_plot(gradient_mtx3, image3, 8, "Q3_3")
generate_quiver_plot(bincounts3, image3, 8, "Q3_3_count")
normalized = get_normalized_histogram(gradient_mtx3, "Q3_3")

(368, 250)
M, N:(46, 31)
(368, 248)
(368, 248)
46 31
46 31


In [16]:
image4 = Image.open("Q3/flash.jpg")
image4 = np.array(image4)
gray4 = cv2.cvtColor(image4, cv2.COLOR_BGR2GRAY)
gradient4, orientation4, gradient_mtx4, bincounts4 = compute_image_gradient_and_threshold(gray4, 100, 0.001)
generate_quiver_plot(gradient_mtx4, image4, 100, "Q3_4")
generate_quiver_plot(bincounts4, image4, 100, "Q3_4_count")

(3024, 4032)
M, N:(30, 40)
(3000, 4000)
(3000, 4000)
30 40
30 40


In [17]:
image5 = Image.open("Q3/noflash.jpg")
image5 = np.array(image5)
gray5 = cv2.cvtColor(image5, cv2.COLOR_BGR2GRAY)
gradient5, orientation5, gradient_mtx5, bincounts5 = compute_image_gradient_and_threshold(gray5, 100, 0.001)
generate_quiver_plot(gradient_mtx5, image5, 100, "Q3_5")
generate_quiver_plot(bincounts5, image5, 100, "Q3_5_count")

(3024, 4032)
M, N:(30, 40)
(3000, 4000)
(3000, 4000)
30 40
30 40


In [18]:
normalized4 = get_normalized_histogram(gradient_mtx4, "Q3_4")
normalized5 = get_normalized_histogram(gradient_mtx5, "Q3_5")

In [19]:
print(np.sum(np.square(gradient_mtx4[:-1, :-1, :] - gradient_mtx5[:-1, :-1, :])))
print(np.sum(np.square(normalized4[:, :, :6] - normalized5[:, :, :6])))

32256318465600.69
60.38666264144986


In [20]:
generate_quiver_plot(normalized4[:, :, :6], image4, 100, "Q3_4_normalized")
generate_quiver_plot(normalized5[:, :, :6], image5, 100, "Q3_5_normalized")

29 39
29 39


# Question 4

In [21]:
def generate_second_moment_matrix(image, ksize=7, sigma=10):
    m, n = image.shape
    eigenvalues = np.zeros((m, n, 2))
    # compute gradients of x and y directions
    Ix = cv2.Sobel(image, cv2.CV_64F, 1, 0)
    Iy = cv2.Sobel(image, cv2.CV_64F, 0, 1)
    IxIy = np.multiply(Ix, Iy)
    Ix2 = np.multiply(Ix, Ix)
    Iy2 = np.multiply(Iy, Iy)
    Ix2_blur = cv2.GaussianBlur(Ix2,ksize=(ksize,ksize), sigmaX=sigma) 
    Iy2_blur = cv2.GaussianBlur(Iy2,ksize=(ksize,ksize), sigmaX=sigma) 
    IxIy_blur = cv2.GaussianBlur(IxIy,ksize=(ksize,ksize), sigmaX=sigma) 
    for i in range(m):
        for j in range(n):
            matrix = np.array([[Ix2_blur[i,j], IxIy_blur[i, j]], [IxIy_blur[i, j], Iy2_blur[i, j]]])
            eigenvalues[i, j] = np.linalg.eigvals(matrix)
    det = np.multiply(Ix2_blur, Iy2_blur) - np.multiply(IxIy_blur,IxIy_blur)
    trace = Ix2_blur + Iy2_blur
    return eigenvalues, det, trace

In [22]:
image1 = Image.open("Q4/csc420Q41.jpeg")
image1 = np.array(image1)
gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
plt.imshow(gray1, cmap="gray")
print(gray1.shape)

(679, 1024)


In [ ]:
smm, det, trace = generate_second_moment_matrix(gray1)
fig = plt.figure()
plt.scatter(smm[:, :, 0].flatten(), smm[:, :, 1].flatten(), s=0.5, alpha=0.1)
fig.savefig("Q4_image1_scatter.png")

In [23]:
image2 = Image.open("Q4/csc420Q42.jpeg")
image2 = np.array(image2)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
plt.imshow(gray2, cmap="gray")

In [ ]:
smm2, det2, trace2 = generate_second_moment_matrix(gray2)
fig = plt.figure()
plt.scatter(smm2[:, :, 0].flatten(), smm2[:, :, 1].flatten(), s=0.5, alpha=0.1)
fig.savefig("Q4_image2_scatter.png")

In [ ]:
img1 = image1.copy()
print(img1.shape)
print(smm.shape)
min_smm = np.min(smm, axis=2)
#img1[R > 0.05*R.max()]=[255, 0, 0]
img1[min_smm > 25000]=[255, 255, 0] 
plt.figure(figsize=(7,7))
# plt.imshow(img1)
plt.imsave("Q4_1.png", img1)

In [ ]:
img2 = image2.copy()
print(img2.shape)
print(smm2.shape)
min_smm2 = np.min(smm2, axis=2)
print(min_smm2.shape)
#img2[R2 > 0.05* R2.max()]=[255, 0, 0] 
img2[min_smm2 > 30000]=[255, 255, 0] 
plt.figure(figsize=(7,7))
plt.imsave("Q4_2.png", img2)

In [30]:
# ksize = [3, 5, 7, 9, 11]
# sigma = [0.5, 1, 2, 3, 7, 20]
ksize=[7]
sigma=[100]
for k in ksize:
    for s in sigma:
        smm, det, trace = generate_second_moment_matrix(gray1, ksize=k, sigma=s)
        fig, ax = plt.subplots()
        ax.scatter(smm[:, :, 0].flatten(), smm[:, :, 1].flatten(), s=0.5, alpha=0.1)
        fig.savefig(f"images/scatter_image1_{k}_{s}.png")
        print(f"scatter_image1_{k}_{s}.png saved")
        smm2, det2, trace2 = generate_second_moment_matrix(gray2, ksize=k, sigma=s)
        fig2, ax2 = plt.subplots()
        ax2.scatter(smm2[:, :, 0].flatten(), smm2[:, :, 1].flatten(), s=0.5, alpha=0.1)
        fig2.savefig(f"images/scatter_image2_{k}_{s}.png")
        print(f"scatter_image2_{k}_{s}.png saved")
        # Image 1
        img1 = image1.copy()
        min_smm = np.min(smm, axis=2)
        img1[min_smm > 25000]=[255, 255, 0] 
        plt.figure(figsize=(7,7))
        plt.imsave(f"images/corners_image1_{k}_{s}.png", img1)
        print(f"corners_image1_{k}_{s}.png saved")
        # Image 2
        img2 = image2.copy()
        min_smm2 = np.min(smm2, axis=2)
        img2[min_smm2 > 30000]=[255, 255, 0] 
        plt.figure(figsize=(7,7))
        plt.imsave(f"images/corners_image2_{k}_{s}.png", img2)
        print(f"corners_image2_{k}_{s}.png saved")
        

<ipython-input-30-fdb3271d7ea7>:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


scatter_image1_7_100.png saved
scatter_image2_7_100.png saved
corners_image1_7_100.png saved
corners_image2_7_100.png saved
